In [ ]:
#Import libraries for simulation
import tensorflow as tf
import numpy as np
sess = tf.InteractiveSession()

In [ ]:
#Imports for visualization
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.animation as animation

def staticPlot(u):
    x = np.linspace(0, 2*np.pi, len(u))
    fig, ax = plt.subplots()
    line, = ax.plot(x, u)
    plt.show()

In [ ]:
def make_kernel(a):
    """Transform a 2D array into a convolution kernel"""
    print("a before reshape = ", a.shape)
    a = a.reshape(list(a.shape) + [1,1])
    print("a after reshape = ", a.shape)
    return tf.constant(a, dtype=np.float64)

def simple_conv(x, k):
    """A simplified 2D convolution operation"""
    print("x=",x)
    x = tf.expand_dims(tf.expand_dims(x, 0), -1)
    print("x for convolution =",x)
    print("k for convolution =",k)
    #y = tf.nn.depthwise_conv2d(x, k, [1, 1, 1, 1], padding='SAME')
    #y = tf.nn.conv2d(x, k, [1, 1, 1, 1], padding='SAME')
    y = tf.nn.conv1d(x, k, 1, padding='SAME')
    print("y=",y)
    return y[0, :, 0]

def laplace(x):
    """Compute the 2D laplacian of an array"""
    constantPutByHand = 0.5*(1./2.)
    #laplace_k = make_kernel([-1./560.,8./315.,-1./5.,8./5.,-205./72.,8./5.,-1./5.,-8./315.,-1./560.])
    laplace_k = make_kernel(constantPutByHand*np.asarray([1.,-2.,1.]))
    
    #laplace_k = make_kernel([[0.25],
    #                         [-0.5],
    #                         [0.25]])
    
    #laplace_k = make_kernel([[0.0, 0.0, 0.0],
    #                       [0.5, -1.0, 0.5],
    #                       [0.0, 0.0, 0.0]])
    print("Laplace k = ", laplace_k)
    return simple_conv(x, laplace_k)


u_test = tf.Variable(np.zeros([100],dtype=np.float64))
f = laplace(u_test)
print("f = ", f)

In [ ]:
# Initial Conditions -- sine wave
#Error in potential energy for the real solution (i.e. sin(x-t)):
#N = 64     # 0.015625
#N = 65     # 0.015384
#N = 100    # 0.010000
#N = 128    # 0.007812
#N = 129    # 0.007751 
#N = 130    # 0.007692
N = 1025    # 0.000975

#N = 16385  # 0.000061 

# Set everything to zero
#u_init = 1e-7*np.ones([N], dtype=np.float32)*np.random.uniform()*np.random.choice([1,-1])
#ut_init = 1e-7*np.ones([N], dtype=np.float32)*np.random.uniform()*np.random.choice([1,-1])
u_init = np.zeros([N], dtype=np.float64)
ut_init = np.zeros([N], dtype=np.float64)
x = np.linspace(0.25*np.pi, 10.25*np.pi, N,dtype=np.float64)
#x = np.linspace(0., 10.0*np.pi, N,dtype=np.float64)

#Changing to this interval increases the error (¿For N = 1025, Error = 0.001953?)
#x = np.linspace(0.0, 2.0*np.pi, N,dtype=np.float32)

for i in range(len(u_init)):
    if(x[i] >= 4*np.pi and x[i] < 6*np.pi):
        u_init[i] = np.sin(x[i])

plot = staticPlot(u_init)

In [ ]:
np.random.uniform()

In [ ]:
# Parameters:
# eps -- time resolution
# damping -- wave damping
eps = tf.placeholder(tf.float64, shape=())
damping = tf.placeholder(tf.float64, shape=())

# Create variables for simulation state
U  = tf.Variable(u_init)
print(U)
Ut = tf.Variable(ut_init)
print(Ut)

# Discretized PDE update rules
U_ = U + eps * Ut
Ut_ = Ut + eps * (laplace(U) - damping * Ut)

# Operation to update the state
step = tf.group(
  U.assign(U_),
  Ut.assign(Ut_))

In [ ]:
# Initialize state to initial conditions
tf.global_variables_initializer().run()

#Do simulation
uEvol = []
sinEvol = []
t0=0
tmax=2048
dt = 1000.0*np.pi/float(tmax)
#With spatial resolution of 1025 elements, the error in the potential energy varying the timesteps is, for the 
#real solution:
#2048 timesteps: 0.00097561
#2049 timesteps: 0.00097561
# 129 timesteps: 0.00097553

for t in range(t0,tmax):
    sinEvol.append(np.sin(x-t*dt))
    uEvol.append(U.eval())
    step.run({eps: dt, damping: 0.00})

In [ ]:
#Check Initial conditions
staticPlot(uEvol[0])
staticPlot(sinEvol[0])
staticPlot(uEvol[0]-sinEvol[0])

In [ ]:
#Imports for visualization
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.animation as animation

fig, (ax1, ax2) = plt.subplots(2)
line1, = ax1.plot(x, uEvol[0],'-b')
line2, = ax2.plot(x, sinEvol[0],'-r')

def animate(t):
    line1.set_ydata(uEvol[t])  # update the data
    line2.set_ydata(sinEvol[t])  # update the data
    ax2.set_xlabel(t)
    return line1, line2

# Init only required for blitting to give a clean slate.
def init():
    line1.set_ydata(uEvol[0])
    line2.set_ydata(sinEvol[0])
    ax1.axhline(1, color='black', lw=1)
    ax2.axhline(1, color='black', lw=1)
    ax1.axhline(-1, color='black', lw=1)
    ax2.axhline(-1, color='black', lw=1)
    ax1.axvline(0.25*np.pi, color='black', lw=1)
    ax2.axvline(0.25*np.pi, color='black', lw=1)
    ax1.axvline(10.25*np.pi, color='black', lw=1)
    ax2.axvline(10.25*np.pi, color='black', lw=1)
    return line1, line2

ani = animation.FuncAnimation(fig, animate, np.arange(t0, tmax), init_func=init,
                              interval=10
                              , blit=True)
plt.show()

In [ ]:
potEnergyTf   = np.asarray([sum([se**2 for se in SE]) for SE in uEvol])
potEnergyReal = np.asarray([sum([se**2 for se in SE]) for SE in sinEvol])


figEnergy, (axEnergyTf,axEnergyReal) = plt.subplots(2)
lineEtf, = axEnergyTf.plot(potEnergyTf/potEnergyTf[0],'-b')
lineEre, = axEnergyReal.plot(potEnergyReal/potEnergyReal[0], '-r')

plt.show()

print(max(potEnergyReal/potEnergyReal[0]))
print(max(potEnergyTf/potEnergyTf[0]))
#plt.plot(uEvol[0])
#plt.show()

len(potEnergyReal)

plt.close()

i=415
plt.semilogy(abs(uEvol[i]))
plt.title(i)

plt.show()


plt.close()
plt.semilogy(abs(uEvol[500][0:200]))
plt.show()

for i in range(len(uEvol)):
    print(i,uEvol[i][0:4])